In [47]:
import datetime as dt
import pandas as pd
import numpy as np

import pandas as pd




In [48]:
btc = pd.read_csv ('btc_limp.csv')
btc_h = pd.read_csv ('Bitcoin_Historical.csv')

btc.set_index('Date', inplace=True)

btc.index = pd.to_datetime(btc.index)



In [49]:
btc_h = btc_h.drop ('Change %', axis=1)

In [50]:
for column in btc_h.columns:
    try:
        btc_h.loc[:, column] = btc_h[column].str.replace(',', '')
    # Convierte la column a tipo float
        btc_h.loc[:, column] = btc_h[column].astype(float)
    # Redondea los valores
        btc_h.loc[:, column] = btc_h[column].round()
    # Convierte los valores redondeados a tipo int
        btc_h.loc[:, column] = btc_h[column].astype(int)
    except ValueError:
        print(f"No se puede convertir la column '{column}' a valores numéricos")

No se puede convertir la column 'Date' a valores numéricos
No se puede convertir la column 'Vol.' a valores numéricos


In [51]:
btc_h['Vol.'] = btc_h['Vol.'].str.rstrip('K')
btc_h['Vol.'] = btc_h['Vol.'].str.rstrip('M')
btc_h['Vol.'] = btc_h['Vol.'].str.rstrip('B')

btc_h['Vol.'] = btc_h['Vol.'].astype(float)



In [52]:

filtro = btc_h['Price'] > 10

btc_h = btc_h[filtro]


In [53]:

btc_h ['Vol.'] = btc_h ['Vol.'] * 1000

btc_h.head ()


,Date,Price,Open,High,Low,Vol.
0,03/18/2024,67594,68389,68920,66601,78070.0
1,03/17/2024,68391,65314,68857,64605,66070.0
2,03/16/2024,65314,69456,70037,64971,75820.0
3,03/15/2024,69463,71387,72398,65765,148590.0
4,03/14/2024,71387,73066,73740,68717,109430.0


In [54]:
btc_h.set_index('Date', inplace=True)

btc_h.index = pd.to_datetime(btc_h.index)


In [55]:
btc_h.rename(columns={'Vol.': 'Vol_BTC'}, inplace=True)

btc_h['Vol_USD'] = btc_h['Price'] * btc_h['Vol_BTC']

btc_h.info ()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4274 entries, 2024-03-18 to 2011-06-03
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Price    4274 non-null   object 
 1   Open     4274 non-null   object 
 2   High     4274 non-null   object 
 3   Low      4274 non-null   object 
 4   Vol_BTC  4268 non-null   float64
 5   Vol_USD  4268 non-null   object 
dtypes: float64(1), object(5)
memory usage: 233.7+ KB


In [56]:
btc_h['Vol_USD'] = (btc_h['Price'] * btc_h['Vol_BTC']).round(2)
btc_h['Vol_USD'] = btc_h['Vol_USD'].apply(lambda x: "{:,.0f}".format(x))


In [57]:
btc_h.to_csv('btc_dia.csv', index=True)

In [58]:
btc_h.tail (5)

,Price,Open,High,Low,Vol_BTC,Vol_USD
Date,,,,,,
2011-06-07,23,18,24,18,53720.0,"1,235,560"
2011-06-06,18,16,19,16,55090.0,"991,620"
2011-06-05,16,18,19,16,24280.0,"388,480"
2011-06-04,18,14,18,14,44480.0,"800,640"
2011-06-03,14,10,14,10,72980.0,"1,021,720"


In [59]:

btc_sem = btc_h.resample('W').first()
btc_sem ['retornos']= btc_sem ['Open'].pct_change ().round (3)

btc_mes = btc_h.resample('ME').first()
btc_mes ['retornos']= btc_mes ['Open'].pct_change ().round (3)




#btc = btc.loc['2014-01-01':]

#btc_week = btc ['Open']

C:\Users\pobla\AppData\Local\Temp\ipykernel_20772\1299966717.py:2: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\pobla\AppData\Local\Temp\ipykernel_20772\1299966717.py:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\pobla\AppData\Local\Temp\ipykernel_20772\1299966717.py:5: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

C:\Users\pobla\AppData\Local\Temp\ipykernel

In [60]:
# Obtener el primer día del año, el precio máximo y mínimo para cada año



In [61]:
btc_ano = btc_h.resample('YE').first()

btc_ano['First_Day'] = btc_h.resample('YS').first().index
btc_ano['Max_High'] = btc_h.resample('YS').max()['High']
btc_ano['Min_Low'] = btc_h.resample('YS').min()['Low']


btc_ano ['retornos']= btc_ano ['Open'].pct_change ().round (3)


btc_ano['Vol_BTC_sum'] = [btc_h[btc_h.index.year == year]['Vol_BTC'].sum() for year in btc_ano.index.year]
btc_ano['Vol_USD_sum'] = [btc_h[btc_h.index.year == year]['Vol_USD'].sum() for year in btc_ano.index.year]

btc_ano.drop(columns=['Vol_BTC'], inplace=True)
btc_ano.drop(columns=['Vol_USD'], inplace=True)

btc_ano



C:\Users\pobla\AppData\Local\Temp\ipykernel_20772\436465891.py:8: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,Price,Open,High,Low,First_Day,Max_High,Min_Low,retornos,Vol_BTC_sum,Vol_USD_sum
Date,,,,,,,,,,
2011-12-31,14,10,14,10,2011-01-01,NaN,NaN,NaN,2508070.0,"95,920109,120370,920168,630419,760686,880282,7..."
2012-12-31,11,10,11,10,2012-01-01,NaN,NaN,0.000,4040830.0,"195,650152,880339,950309,530257,660205,920171,..."
2013-12-31,13,13,13,13,2013-01-01,NaN,NaN,0.300,18007040.0,"7,381,8508,425,9205,526,40010,165,08011,153,67..."
2014-12-31,815,805,829,771,2014-01-01,NaN,NaN,60.923,4412190.0,"6,519,0007,084,5806,393,0406,449,5606,813,4507..."
2015-12-31,314,318,321,313,2015-01-01,NaN,NaN,-0.605,27580980.0,"28,040,30029,151,29030,450,15028,775,35028,518..."
2016-12-31,434,430,438,425,2016-01-01,NaN,NaN,0.352,19602750.0,"26,540,28067,935,56059,357,23075,125,88045,963..."
2017-12-31,995,963,1001,956,2017-01-01,NaN,NaN,1.240,37438920.0,"1,541,089,5002,281,519,1701,710,920,9602,452,9..."
2018-12-31,13444,13850,13921,12877,2018-01-01,NaN,NaN,13.382,75165960.0,"2,024,483,4701,980,633,5501,873,049,4602,182,3..."
2019-12-31,3809,3709,3814,3664,2019-01-01,NaN,NaN,-0.732,186913690.0,"4,221,173,6004,400,964,7104,524,670,9304,472,8..."


In [62]:
print(btc_h[btc_h.index.month == 1].head())

            Price   Open   High    Low  Vol_BTC        Vol_USD
Date                                                          
2024-01-31  42580  42946  43739  42315  56480.0  2,404,918,400
2024-01-30  42946  43303  43817  42702  55130.0  2,367,612,980
2024-01-29  43299  42031  43305  41824  45230.0  1,958,413,770
2024-01-28  42030  42121  42817  41649  32530.0  1,367,235,900
2024-01-27  42120  41811  42191  41413  20460.0    861,775,200


In [63]:
# Verificar si los índices están alineados correctamente
print(btc_h.index)
print(btc_ano.index)


DatetimeIndex(['2024-03-18', '2024-03-17', '2024-03-16', '2024-03-15',
               '2024-03-14', '2024-03-13', '2024-03-12', '2024-03-11',
               '2024-03-10', '2024-03-09',
               ...
               '2011-06-12', '2011-06-11', '2011-06-10', '2011-06-09',
               '2011-06-08', '2011-06-07', '2011-06-06', '2011-06-05',
               '2011-06-04', '2011-06-03'],
              dtype='datetime64[ns]', name='Date', length=4274, freq=None)
DatetimeIndex(['2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31',
               '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31',
               '2019-12-31', '2020-12-31', '2021-12-31', '2022-12-31',
               '2023-12-31', '2024-12-31'],
              dtype='datetime64[ns]', name='Date', freq='YE-DEC')


In [64]:
btc_sem.to_csv('btc_sem.csv', index=True)
btc_mes.to_csv('btc_mes.csv', index=True)
btc_ano.to_csv('btc_ano.csv', index=True)

In [65]:
ruta_archivo = 'btc_sem.csv' 

bitcoin = pd.read_csv(ruta_archivo)


In [66]:
valores_faltantes = btc_sem['Price'].isnull().sum()
print("Valores faltantes en la columna Price:", valores_faltantes)

indices_faltantes = btc_sem[btc_sem['Price'].isnull()].index
print("Índices de los valores faltantes en la columna Price:", indices_faltantes)


Valores faltantes en la columna Price: 53
Índices de los valores faltantes en la columna Price: DatetimeIndex(['2011-08-14', '2011-08-28', '2011-09-04', '2011-09-11',
               '2011-09-18', '2011-09-25', '2011-10-02', '2011-10-09',
               '2011-10-16', '2011-10-23', '2011-10-30', '2011-11-06',
               '2011-11-13', '2011-11-20', '2011-11-27', '2011-12-04',
               '2011-12-11', '2011-12-18', '2011-12-25', '2012-01-01',
               '2012-01-08', '2012-01-15', '2012-01-22', '2012-01-29',
               '2012-02-05', '2012-02-12', '2012-02-19', '2012-02-26',
               '2012-03-04', '2012-03-11', '2012-03-18', '2012-03-25',
               '2012-04-01', '2012-04-08', '2012-04-15', '2012-04-22',
               '2012-04-29', '2012-05-06', '2012-05-13', '2012-05-20',
               '2012-05-27', '2012-06-03', '2012-06-10', '2012-06-17',
               '2012-06-24', '2012-07-01', '2012-07-08', '2012-07-15',
               '2012-07-22', '2012-07-29', '2012-08-

In [67]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

btc_h_2 = btc_h.loc[btc_h.index.year == 2015]


In [68]:
# Filtrar por el año 2014
btc_h_2014 = btc_h.loc[btc_h.index.year == 2014]

# Encontrar el valor mínimo en la columna 'Low'
min_low_2014 = btc_h_2014['Low'].min()

print("El número mínimo en la columna 'Low' para el año 2014 es:", min_low_2014)


El número mínimo en la columna 'Low' para el año 2014 es: 91
